In [1]:
import climate_functions as cf
import numpy as np
import synthetic_data as sd
import network_builder as nb
import networkx as nx
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
#from fastdtw import fastdtw
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis
#from dtaidistance import clustering
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
sns.set()
import random
import pandas as pd
from netCDF4 import Dataset
from stldecompose import decompose, forecast
import folium
import math
import scipy.stats as st
#import cufflinks as cf
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='mohammad5617', api_key='yElRsXEjk0sRoqCmfVBn')
import plotly.io as pio
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

/home/mathsys2/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
f_pre = Dataset('precipitation.nc')
pr = f_pre.variables['precip']
lon = f_pre.variables['lon'][:]
lat = f_pre.variables['lat'][:]
pr = np.swapaxes(pr,0,2)

r = np.arange(0.05,1,0.05)

In [3]:
#data = pr[10:22,3:11,:]
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

# Cross validation

In [ ]:
data_deseasonal= cf.deseasonal_STL(data)
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)
data_deseasonal_monthly= cf.deseasonal_monthly_anomaly(data)
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

upper_bound = go.Scatter(
    name='Upper Bound',
    x=r,
    y=r2_upper,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty'
)

lower_bound = go.Scatter(
    name='Lower Bound',
    x=r,
    y=r2_lower,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0)
)

In [4]:
MSEds_list = []
MSE_list = []
MSEdm_list = []
null_list =[]
for i in range(0,840,168):
    MSE, average_betweennes, average_shortest_path_length,density = cf.model(data,"LR","normal",start=i,end=i+168, data_type="normal")
    MSE_list.append(MSE)
    
    MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, density_ds = cf.model(data,"LR","normal",start=i,end=i+168, data_type="STL")
    MSEds_list.append(MSE_ds)

    MSE_dm, average_betweennes_dm, average_shortest_path_length_dm,density_dm = cf.model(data,"LR","normal",start=i,end=i+168, data_type="PA")
    MSEdm_list.append(MSE_dm)
    
 #   null = cf.exponential_smoothing(data[i:i+168-1,:], 0.95)
 #   error = mean_squared_error(data[i+168-1,:], null)
 #   null_list.append(error)

In [64]:
#f = np.matrix(r2_list).std(axis=0)/math.sqrt(20) * 1.96
#r2_list_bounds = np.reshape(f,(18,1))
#r2_list_bounds = r2_list_bounds.tolist()[0]


In [5]:
#null = np.mean(null_list)
#nullr2ds = np.mean(nullds_list)

In [5]:
f = np.matrix(MSE_list).mean(axis=0)
MSE = np.reshape(f,(19,1))

f1 = np.matrix(MSEds_list).mean(axis=0)
MSE_ds = np.reshape(f1,(19,1))

f2 = np.matrix(MSEdm_list).mean(axis=0)
MSE_dm = np.reshape(f2,(19,1))

#r_rev = r[::-1]#

#r2_upper = r2 + r2_list_bounds
#r2_lower = r2 - r2_list_bounds

In [12]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'Original data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'STL decomposition',
    line = dict(
        color = ('rgb(22, 167, 40)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dm,
    name = 'Phase Averaging',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)


d = [trace0,trace1, trace2]
    
layout = dict(title = 'Models Performance',
              height=800,
              width = 800,
              xaxis = dict(title = 'Thresholds',              
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,),
              yaxis = dict(title = 'Mean Square Error',
              range = [0,800]            )
       )

fig = dict(data=d, layout=layout)
pio.write_image(fig, 'performance_cross.png')
py.iplot(fig, filename='styled-line')

In [19]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'Original data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'STL decomposition',
    line = dict(
        color = ('rgb(22, 167, 40)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dm,
    name = 'Phase Averaging',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = np.repeat(error, 19),
    name = 'Exponential smoothing',
    line = dict(
        color = ('rgb(100, 10, 240)'),
        width = 0.5, 
        dash = 'longdash')
)

d = [trace0,trace1, trace2, trace3]
    
layout = dict(title = 'Models Performance',
              height=800,
              width = 800,
              xaxis = dict(title = 'Thresholds',              
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,),
              yaxis = dict(title = 'Mean Square Error')
       )

fig = dict(data=d, layout=layout)
pio.write_image(fig, 'performance_cross.png')
py.iplot(fig, filename='styled-line')


In [16]:

trace0 = go.Scatter(
    x = r ,
    y = MSE,
    name = 'normal data',
    mode = 'lines',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,dash = 'dot'),
   # fillcolor='rgba(68, 68, 68, 0.3)',
    #fill='tonexty'
)

trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)   
trace2 = go.Scatter(
    x = r,
    y = MSE_dm,
    name = 'PA deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)



d = [trace0, trace1]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              width=1000,
              height=800,
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000')
            
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

# Matrix Distribution

In [19]:
end = 1
start= 0 

In [20]:
data_deseasonal = data
data_seasonal = np.zeros(data.shape[1]) 
m = nb.weighted_matrix(data_deseasonal[start:-end-1,:])

In [22]:
h = cf.flatten2d(np.absolute(m))
layout = dict(title = 'Distribution of edge weights made from original data')
d = [go.Histogram(x=h)]
fig = go.Figure(data=d, layout=layout)
pio.write_image(fig, 'distribution_normal.png')
py.iplot(fig, filename='basic histogram')

In [31]:
#data = pr[10:22,3:11,:]
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [32]:
data_deseasonal, data_seasonal = cf.deseasonal_monthly_anomaly(data)
data_seasonal = data_seasonal[0,:]
m = nb.weighted_matrix(data_deseasonal[start:-end-1,:])

In [33]:
h = cf.flatten2d(np.absolute(m))
layout = dict(title = 'Distribution of edge weights made from phase averaging deseasonal data')
d = [go.Histogram(x=h)]
fig = go.Figure(data=d, layout=layout)
pio.write_image(fig, 'distribution_pa.png')
py.iplot(d, filename='basic histogram')

In [28]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [29]:
data_deseasonal, data_seasonal = cf.deseasonal_STL(data)
data_seasonal = data_seasonal[-end,:]
m = nb.weighted_matrix(data_deseasonal[start:-end-1,:])

In [30]:
h = cf.flatten2d(np.absolute(m))
layout = dict(title = 'Distribution of edge weights made from STL deseasonal data')
d = [go.Histogram(x=h)]
fig = go.Figure(data=d, layout=layout)
pio.write_image(fig, 'distribution_STL.png')
py.iplot(d, filename='basic histogram')

# Linear Regression

In [17]:
MSE, average_betweennes, average_shortest_path_length, density = cf.model(data,"LR","normal")

In [18]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)
MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, density_ds = cf.model(data,"LR","normal",data_type = "STL")

In [19]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)
MSE_dsm, average_betweennes_dsm, average_shortest_path_length_dsm, density_dm = cf.model(data,"LR","normal",data_type = "PA")

In [8]:
null = cf.exponential_smoothing(data, 0.95)
error = mean_squared_error(data[-end,:], null)

In [10]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'Original data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'STL decomposition',
    line = dict(
        color = ('rgb(22, 167, 40)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dsm,
    name = 'Phase Averaging',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = np.repeat(error, 19),
    name = 'Exponential smoothing',
    line = dict(
        color = ('rgb(100, 10, 240)'),
        width = 0.5, 
        dash = 'longdash')
)

d = [trace0,trace1, trace2, trace3]
    
layout = dict(title = 'Models Performance',
              height=800,
              width = 800,
              xaxis = dict(title = 'Thresholds',              
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,),
              yaxis = dict(title = 'Mean Square Error',range=[0,200])
       )

fig = dict(data=d, layout=layout)
pio.write_image(fig, 'performance.png')
py.iplot(fig, filename='styled-line')


In [10]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'Original data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'STL decomposition',
    line = dict(
        color = ('rgb(22, 167, 40)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dsm,
    name = 'Phase Averaging',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = np.repeat(error, 19),
    name = 'Exponential smoothing',
    line = dict(
        color = ('rgb(100, 10, 240)'),
        width = 0.5, 
        dash = 'longdash')
)

d = [trace0,trace1, trace2, trace3]
    
layout = dict(title = 'Models Performance',
              height=800,
              width = 800,
              xaxis = dict(title = 'Thresholds',              
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,),
              yaxis = dict(title = 'Mean Square Error',range=[0,800])
       )

fig = dict(data=d, layout=layout)
pio.write_image(fig, 'performance.png')
py.iplot(fig, filename='styled-line')


In [15]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'MSE of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'MSE of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dsm,
    name = 'MSE of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              height=800,
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
             # dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000')
            
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [21]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'MSE of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'MSE of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dsm,
    name = 'MSE of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              height=800,
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,

              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
       )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [11]:
trace0 = go.Scatter(
    x = r,
    y = average_shortest_path_length,
    name = 'Average Shortest Path of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2)
)
trace1 = go.Scatter(
    x = r,
    y = average_shortest_path_length_ds,
    name = 'Average Shortest Path of deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,)
) 

trace2 = go.Scatter(
    x = r,
    y = average_shortest_path_length_dsm,
    name = 'Average Shortest Path of deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 2,
    dash = 'dash')
)
 
d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of Average Shortest Path',
              xaxis = dict(title = 'Thresholds'),
              yaxis = dict(title = 'Average Shortest Path'),
 #             shapes = [
 #       {
 #           'type': 'line',
 #           'x0': 0.35,
 #           'y0': 0,
 #           'x1': 0.35,
 #           'y1': 3.5,
  #          'line': {
   #             'color': 'rgb(0, 0, 0)',
   #             'width': 0.5,
   #         },
    #    }]
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [13]:
trace0 = go.Scatter(
    x = r,
    y = average_betweennes,
    name = 'Average Betweenness of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2)
)
trace1 = go.Scatter(
    x = r,
    y = average_betweennes_ds,
    name = 'Average Betweenness of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,)
)    
trace2 = go.Scatter(
    x = r,
    y = average_betweennes_dsm,
    name = 'Average Betweenness of Anomalydeseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 2,
    dash = 'dash')
)
d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of Betweenness',
              xaxis = dict(title = 'Thresholds'),
              yaxis = dict(title = 'Average Betweennes'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [14]:
density = np.array(density)
density_ds = np.array(density_ds)
density_dm = np.array(density_dm)

In [15]:
trace0 = go.Scatter(
    x = r,
    y = density,
    name = 'confidence of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2)
)
trace1 = go.Scatter(
    x = r,
    y = density_ds,
    name = 'confidence of deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,)
) 
trace2 = go.Scatter(
    x = r,
    y = density_ds,
    name = 'confidence of deseasonal data',
    line = dict(
        color = ('rgb(22, 196, 067)'),
        width = 2,)
)
d = [trace0, trace1, trace2]
    

layout = dict(title = 'Comparison of Confidence',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Confidence',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000')
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

# Neural Network

In [78]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [79]:
R2, MSE, average_betweennes, average_shortest_path_length, error, null_r2, density = cf.model(data,"NN","normal")

data_deseasonal= cf.deseasonal_STL(data)
R2_ds, MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, error_ds, null_r2_ds, density_ds = cf.model(data_deseasonal,"NN","normal")

data_deseasonal_monthly= cf.deseasonal_monthly_anomaly(data)
R2_dsm, MSE_dsm, average_betweennes_dsm, average_shortest_path_length_dsm, error_dsm, null_r2_dsm, density_dm = cf.model(data_deseasonal_monthly,"NN","normal")


In [80]:
trace0 = go.Scatter(
    x = r,
    y = R2,
    name = 'R2 of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = R2_ds,
    name = 'R2 of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = R2_dsm,
    name = 'R2 of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

d = [trace0, trace1]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

# Random Forest

In [17]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [19]:
MSE, average_betweennes, average_shortest_path_length,density,b = cf.model(data,"RF","normal")


MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, density_ds,b = cf.model(data,"RF","normal",data_type = "STL")

MSE_dsm, average_betweennes_dsm, average_shortest_path_length_dsm, density_dm,b = cf.model(data,"RF","normal",data_type = "PA")

In [26]:
trace0 = go.Scatter(
    x = r,
    y = MSE,
    name = 'Original data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = MSE_ds,
    name = 'STL decomposition',
    line = dict(
        color = ('rgb(22, 167, 40)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = MSE_dsm,
    name = 'Phase Averaging',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = np.repeat(error, 19),
    name = 'Exponential smoothing',
    line = dict(
        color = ('rgb(100, 10, 240)'),
        width = 0.5, 
        dash = 'longdash')
)

d = [trace0,trace1, trace2, trace3]
    
layout = dict(title = 'Models Performance',
              height=800,
              width = 800,
              xaxis = dict(title = 'Thresholds',              
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,),
              yaxis = dict(title = 'Mean Square Error',range=[100,800])
       )

fig = dict(data=d, layout=layout)
pio.write_image(fig, 'performance_RF.png')
py.iplot(fig, filename='styled-line')


In [33]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [34]:
r = np.arange(0.1,1,0.05)
df = cf.feature_importance(data)

In [35]:
trace0 = go.Scatter(
    x = r,
    y = df['degree'],
    name = 'degree',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,
    dash = 'solid')
)
 
trace2 = go.Scatter(
    x = r,
    y = df['closseness'],
    name = 'closseness',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = df['kcore'],
    name = 'kcore',
    line = dict(
        color = ('rgb(202, 125, 68)'),
        width = 4,
        dash = 'dot')
)
trace4 = go.Scatter(
    x = r,
    y = df['betweeness'],
    name = 'betweeness',
    line = dict(
        color = ('rgb(105, 184, 68)'),
        width = 4,
        dash = 'dashdot')
)
trace5 = go.Scatter(
    x = r,
    y = df['pagerank'],
    name = 'pagerank',
    line = dict(
        color = ('rgb(3, 1, 5)'),
        width = 4,
        dash = 'longdashdot')
)
trace6 = go.Scatter(
    x = r,
    y = df['eigenvector'],
    name = 'eigenvector',
    line = dict(
        color = ('rgb(239, 38, 205)'),
        width = 4,
        dash = 'longdash')
)
trace7 = go.Scatter(
    x = r,
    y = df['clustering'],
    name = 'clustering',
    line = dict(
        color = ('rgb(248, 215, 34)'),
        width = 4,
        dash = 'dash')
)
d = [trace0, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(title = 'Features Importance using Original Data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Importance',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [36]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [38]:
data_deseasonal, data_seasonal= cf.deseasonal_STL(data)
df = cf.feature_importance(data_deseasonal)

trace0 = go.Scatter(
    x = r,
    y = df['degree'],
    name = 'degree',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,
    dash = 'solid')
)
 
trace2 = go.Scatter(
    x = r,
    y = df['closseness'],
    name = 'closseness',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = df['kcore'],
    name = 'kcore',
    line = dict(
        color = ('rgb(202, 125, 68)'),
        width = 4,
        dash = 'dot')
)
trace4 = go.Scatter(
    x = r,
    y = df['betweeness'],
    name = 'betweeness',
    line = dict(
        color = ('rgb(105, 184, 68)'),
        width = 4,
        dash = 'dashdot')
)
trace5 = go.Scatter(
    x = r,
    y = df['pagerank'],
    name = 'pagerank',
    line = dict(
        color = ('rgb(3, 1, 5)'),
        width = 4,
        dash = 'longdashdot')
)
trace6 = go.Scatter(
    x = r,
    y = df['eigenvector'],
    name = 'eigenvector',
    line = dict(
        color = ('rgb(239, 38, 205)'),
        width = 4,
        dash = 'longdash')
)
trace7 = go.Scatter(
    x = r,
    y = df['clustering'],
    name = 'clustering',
    line = dict(
        color = ('rgb(248, 215, 34)'),
        width = 4,
        dash = 'dash')
)
d = [trace0, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(title = 'Features Importance using STL deseasonal Data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Importance',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [41]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [42]:
data_deseasonal, data_seasonal= cf.deseasonal_monthly_anomaly(data)
df = cf.feature_importance(data_deseasonal)

trace0 = go.Scatter(
    x = r,
    y = df['degree'],
    name = 'degree',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,
    dash = 'solid')
)
 
trace2 = go.Scatter(
    x = r,
    y = df['closseness'],
    name = 'closseness',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = df['kcore'],
    name = 'kcore',
    line = dict(
        color = ('rgb(202, 125, 68)'),
        width = 4,
        dash = 'dot')
)
trace4 = go.Scatter(
    x = r,
    y = df['betweeness'],
    name = 'betweeness',
    line = dict(
        color = ('rgb(105, 184, 68)'),
        width = 4,
        dash = 'dashdot')
)
trace5 = go.Scatter(
    x = r,
    y = df['pagerank'],
    name = 'pagerank',
    line = dict(
        color = ('rgb(3, 1, 5)'),
        width = 4,
        dash = 'longdashdot')
)
trace6 = go.Scatter(
    x = r,
    y = df['eigenvector'],
    name = 'eigenvector',
    line = dict(
        color = ('rgb(239, 38, 205)'),
        width = 4,
        dash = 'longdash')
)
trace7 = go.Scatter(
    x = r,
    y = df['clustering'],
    name = 'clustering',
    line = dict(
        color = ('rgb(248, 215, 34)'),
        width = 4,
        dash = 'dash')
)
d = [trace0, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(title = 'Features Importance using Anomaly deseasonal Data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Importance',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [8]:
trace = go.Scatter(
    x = r,
    y = MSE
)

data = [trace]
layout = {}
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='line-mode')